

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/CLASSIFICATION_BIOMARKER.ipynb)


# Bert For Sequence Classification (Biomarker)

This model is a sentence classification system based on `BioBERT` that is capable of identifying if clinical sentences contain terms associated with biomarkers.

| Label | Meaning                                 |
|-------|-----------------------------------------|
| 1     | Contains biomarker related terms        |
| 0     | Doesn't contain biomarker related terms |

-----------------

> 📌To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

## 🔧1. Colab Setup

- Import License Keys

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.2.2
Spark NLP_JSL Version : 5.2.1


## 🔍2. Select the model `bert_sequence_classifier_biomarker` and construct the pipeline

**🔎You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?edition=Spark+NLP+for+Healthcare)**

In [37]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(['sentence']) \
    .setOutputCol('token')

sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_biomarker","en","clinical/models")\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    sequenceClassifier
])


sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
bert_sequence_classifier_biomarker download started this may take some time.
[OK!]


## 📝3. Create example inputs

In [38]:
text_list = [
    """In the realm of cancer research, several biomarkers have emerged as crucial indicators of disease progression and treatment response.""",
    """For instance, the expression levels of HER2/neu, a protein receptor, have been linked to aggressive forms of breast cancer.""",
    """Additionally, the presence of prostate-specific antigen (PSA) is often monitored to track the progression of prostate cancer.""",
    """Moreover, in cardiovascular health, high-sensitivity C-reactive protein (hs-CRP) serves as a biomarker for inflammation and potential risk of heart disease.""",
    """Meanwhile, elevated levels of troponin T are indicative of myocardial damage, commonly observed in acute coronary syndrome.""",
    """In the field of diabetes management, glycated hemoglobin is a widely used to assess long-term blood sugar control.""",
    """Its levels reflect the average blood glucose concentration over the past two to three months, offering valuable insights into disease management strategies."""
]

## 🚀4. Run the pipeline to get the labels

| Label | Meaning                                 |
|-------|-----------------------------------------|
| 1     | Contains biomarker related terms        |
| 0     | Doesn't contain biomarker related terms |


In [39]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
results = pipeline.fit(df).transform(df)

res = results.select(F.explode(F.arrays_zip(results.document.result,
                                             results.class_.result,
                                             results.class_.metadata)).alias("col"))\
               .select(F.expr("col['1']").alias("prediction"),
                       F.expr("col['2']").alias("confidence"),
                       F.expr("col['0']").alias("sentence"))

if res.count()>1:
  udf_func = F.udf(lambda x,y:  x["Some("+str(y)+")"])
  res.withColumn('confidence', udf_func(res.confidence, res.prediction)).show(truncate=150)

+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction|confidence|                                                                                                                                              sentence|
+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|         0|0.99979603|                 In the realm of cancer research, several biomarkers have emerged as crucial indicators of disease progression and treatment response.|
|         1| 0.9732248|                           For instance, the expression levels of HER2/neu, a protein receptor, have been linked to aggressive forms of breast cancer.|
|         1| 0.9997541|                         Additionally, the presence of prostate-specific antigen (PSA) is often monito